In [3]:
import pandas as pd

In [4]:
jan_df = pd.read_csv('data/Jan Sold With.csv')
nov_df = pd.read_csv('data/Nov Sold With.csv')
dec_df = pd.read_csv('data/Dec Sold With.csv')
stores_df = pd.read_csv('data/stores.csv')
store_sales_df = pd.read_csv('data/Store Sales.csv')
products_df = pd.read_csv('data/products.csv')

C:\Users\alecb\AppData\Local\Temp\ipykernel_98896\1282520034.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  jan_df = pd.read_csv('data/Jan Sold With.csv')
C:\Users\alecb\AppData\Local\Temp\ipykernel_98896\1282520034.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  nov_df = pd.read_csv('data/Nov Sold With.csv')
C:\Users\alecb\AppData\Local\Temp\ipykernel_98896\1282520034.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dec_df = pd.read_csv('data/Dec Sold With.csv')


#### I am concatting all the sales tables (nov. ,dec., and jan.) into one df called sales_df

In [5]:
sales_df = pd.concat([nov_df, dec_df, jan_df])
sales_df.head(3)

,DIV,STORE,TRANSACTION_DATE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_EMP_IND,PRIMARY_SOS_IND,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_EMP_IND,SECONDARY_SOS_IND,SECONDARY_UNITS,PRIMARY_LINENO,SECONDARY_LINENO
0,8,12,02-NOV-22,99497860,080,M,N,Y,1,497403,060,M,N,N,1,1,3
1,8,12,02-NOV-22,497403,060,M,N,N,1,99497860,080,M,N,Y,1,3,1
2,8,12,03-NOV-22,398562,100,M,N,N,1,19133,XL,NaN,N,N,1,2,1


I am now joining the DEPARTMENT from the products table to the January sales table so I can filter on accessory or shoe. I will be joining on STOCKNO and PRIMARY_STOCKNO.

In [6]:
master_sale_df1 = pd.merge(sales_df, products_df[['STOCKNO','DEPARTMENT']], 
                               left_on = 'PRIMARY_STOCKNO', right_on = 'STOCKNO', 
                               how = 'left')

In [7]:
master_sale_df1 = master_sale_df1.rename(columns={"DEPARTMENT": "PRIMARY_DEPT"})

In [8]:
master_sale_df1.head(3)

,DIV,STORE,TRANSACTION_DATE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_EMP_IND,PRIMARY_SOS_IND,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_EMP_IND,SECONDARY_SOS_IND,SECONDARY_UNITS,PRIMARY_LINENO,SECONDARY_LINENO,STOCKNO,PRIMARY_DEPT
0,8,12,02-NOV-22,99497860,080,M,N,Y,1,497403,060,M,N,N,1,1,3,99497860.0,7.0
1,8,12,02-NOV-22,497403,060,M,N,N,1,99497860,080,M,N,Y,1,3,1,497403.0,1.0
2,8,12,03-NOV-22,398562,100,M,N,N,1,19133,XL,NaN,N,N,1,2,1,398562.0,3.0


#### Doing the same thing but for SECONDARY_STOCKNO

In [9]:
master_sale_df = pd.merge(master_sale_df1, products_df[['STOCKNO','DEPARTMENT']], 
                               left_on = 'SECONDARY_STOCKNO', right_on = 'STOCKNO', 
                               how = 'left')

In [10]:
master_sale_df = master_sale_df.rename(columns={"DEPARTMENT": "SECONDARY_DEPT"})

#### Dropping some columns

In [11]:
master_sale_df = master_sale_df.drop(columns = ['TRANSACTION_DATE', 'PRIMARY_EMP_IND',
                                                              'SECONDARY_EMP_IND', 'PRIMARY_SOS_IND',
                                                              'SECONDARY_SOS_IND', 'PRIMARY_LINENO',
                                                              'SECONDARY_LINENO', 'STOCKNO_x',
                                                              'STOCKNO_y' ])

In [12]:
master_sale_df.head(5)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_UNITS,PRIMARY_DEPT,SECONDARY_DEPT
0,8,12,99497860,080,M,1,497403,060,M,1,7.0,1.0
1,8,12,497403,060,M,1,99497860,080,M,1,1.0,7.0
2,8,12,398562,100,M,1,19133,XL,NaN,1,3.0,6.0
3,8,12,1497835,130,M,1,398564,070,M,1,4.0,3.0
4,8,12,1497835,130,M,1,398564,060,M,1,4.0,3.0


#### Creating a new column PRIMARY_SHOE (True or False). This will be used to distinguish a shoe from an accessory

In [13]:
for index, row in master_sale_df.iterrows():
    if row.PRIMARY_DEPT in [1, 2, 3, 4, 7]:
        master_sale_df.loc[index, 'PRIMARY_SHOE'] = 'True'
    elif row.PRIMARY_DEPT in [5, 6, 8, 9, 10, 15]:
        master_sale_df.loc[index, 'PRIMARY_SHOE'] = 'False'
    else:
        master_sale_df.loc[index, 'PRIMARY_SHOE'] = 'error'

#### Creating another column SECONDARY_SHOE that provides the same insight as PRIMARY_SHOE.

In [14]:
for index, row in master_sale_df.iterrows():
    if row.SECONDARY_DEPT in [1, 2, 3, 4, 7]:
        master_sale_df.loc[index, 'SECONDARY_SHOE'] = 'True'
    elif row.SECONDARY_DEPT in [5, 6, 8, 9, 10, 15]:
        master_sale_df.loc[index, 'SECONDARY_SHOE'] = 'False'
    else:
        master_sale_df.loc[index, 'SECONDARY_SHOE'] = 'error'

#### I want to be able to filter on location hierarchy so I am joining the stores table to the sales table using 'store'.

In [15]:
master_sale_df = pd.merge(master_sale_df, stores_df[['DIV','STORE','REGION','DISTRICT','STORE_STATE_ALPHA',
                                                     'STORE_ZIP','LATITUDE','LONGITUDE']], 
                               left_on = ('STORE', 'DIV'), right_on = ('STORE', 'DIV'), 
                               how = 'left')

#### Creating a table for shoes sold with shoes (shoe_shoe_sale), and a table for shoes sold with accessories (shoe_acces_sale)

In [16]:
shoe_shoe_sale = master_sale_df[(master_sale_df['SECONDARY_SHOE'] == 'True') 
                                      & (master_sale_df['PRIMARY_SHOE'] == 'True')]

shoe_acces_sale = master_sale_df[(master_sale_df['SECONDARY_SHOE'] == 'False')
                                      & (master_sale_df['PRIMARY_SHOE'] == 'True')]

### The following functions are used to find the top 3 shoe or accessory recomendations given a shoe's stock number. The first list contains the recomendations. The second list returns the amount of times this combonation has been purchased.

In [17]:
def shoe_recomendations(stock_num):
    
    recomendation_df = pd.DataFrame(shoe_shoe_sale[(shoe_shoe_sale['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count

In [18]:
def acces_recomendations(stock_num):
    
    recomendation_df = pd.DataFrame(shoe_acces_sale[(shoe_acces_sale['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count

In [17]:
shoe_recomendations(398913)

([398564, 398732, 398566], [3701, 660, 592])

In [18]:
acces_recomendations(398913)

([1639, 401, 434], [5034, 2289, 805])

In [19]:
shoe_shoe_sale.head(3)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_UNITS,PRIMARY_DEPT,SECONDARY_DEPT,PRIMARY_SHOE,SECONDARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA,STORE_ZIP,LATITUDE,LONGITUDE
0,8,12,99497860,080,M,1,497403,060,M,1,7.0,1.0,True,True,9.0,29.0,MI,483131320.0,42.6019,-82.998
1,8,12,497403,060,M,1,99497860,080,M,1,1.0,7.0,True,True,9.0,29.0,MI,483131320.0,42.6019,-82.998
3,8,12,1497835,130,M,1,398564,070,M,1,4.0,3.0,True,True,9.0,29.0,MI,483131320.0,42.6019,-82.998


#### Creating similiar functions that can look into different location hierarchies.

In [19]:
all_regions = shoe_shoe_sale.REGION.unique()
all_districts = shoe_shoe_sale.DISTRICT.unique()
all_states = shoe_shoe_sale.STORE_STATE_ALPHA.unique()
all_stores = shoe_shoe_sale.STORE.unique()

In [20]:
def shoe_recomendations2(stock_num, region = all_regions, district = all_districts,
                         state = all_states, store = all_stores):
    
    shoe_shoe = shoe_shoe_sale[(shoe_shoe_sale['REGION'].isin(region)) &
                                    (shoe_shoe_sale['DISTRICT'].isin(district)) &
                                    (shoe_shoe_sale['STORE_STATE_ALPHA'].isin(state)) &
                                    (shoe_shoe_sale['STORE'].isin(store))]
    if len(store) == 1:
        heirarchy ='store'
    elif len(state) == 1:
        heirarchy ='state'
    elif len(district) == 1:
        heirarchy ='district'
    elif len(region) == 1:
        heirarchy ='region'
    else:
        heirarchy = 'division'
    
    recomendation_df = pd.DataFrame(shoe_shoe[(shoe_shoe['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count, heirarchy

In [22]:
shoe_recomendations2(99497860)

([1497860, 99497858, 99498066], [70, 16, 14], 'division')

In [21]:
def acces_recomendations2(stock_num, region = all_regions, district = all_districts,
                         state = all_states, store = all_stores):
    
    acces_acces = shoe_acces_sale[(shoe_acces_sale['REGION'].isin(region)) &
                                    (shoe_acces_sale['DISTRICT'].isin(district)) &
                                    (shoe_acces_sale['STORE_STATE_ALPHA'].isin(state)) &
                                    (shoe_acces_sale['STORE'].isin(store))]
    
    if len(store) == 1:
        heirarchy ='store'
    elif len(state) == 1:
        heirarchy ='state'
    elif len(district) == 1:
        heirarchy ='district'
    elif len(region) == 1:
        heirarchy ='region'
    else:
        heirarchy = 'division'
    
    recomendation_df = pd.DataFrame(acces_acces[(acces_acces['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count, heirarchy

In [24]:
acces_recomendations2(99497860)

([1829, 3875, 1018], [31, 24, 22], 'division')

### I would like to apply the functions I created to the sales table (shoe&shoe sales, shoe&accessory sales). My desired result is a table with the store number, product number, 6 recomendations (3 shoes, 3 accessories), and at what heirarchy level the recomendations come from (store, region, state, etc.)

Setting up a tables to be filled

In [47]:
shoes_df = shoe_shoe_sale.reset_index()
shoes_df = shoes_df.drop(columns = ['index', 'PRIMARY_UNITS', 'SECONDARY_UNITS',
                              'SECONDARY_ITEM_SIZE', 'SECONDARY_ITEM_SIZE', 'SECONDARY_ITEM_WIDTH',
                              'SECONDARY_SHOE', 'STORE_ZIP',
                              'LATITUDE', 'LONGITUDE'])
shoes_df.head()

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,SECONDARY_STOCKNO,PRIMARY_DEPT,SECONDARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA
0,8,12,99497860,080,M,497403,7.0,1.0,True,9.0,29.0,MI
1,8,12,497403,060,M,99497860,1.0,7.0,True,9.0,29.0,MI
2,8,12,1497835,130,M,398564,4.0,3.0,True,9.0,29.0,MI
3,8,12,1497835,130,M,398564,4.0,3.0,True,9.0,29.0,MI
4,8,12,398564,070,M,1497835,3.0,4.0,True,9.0,29.0,MI


In [53]:
shoes_nation_level_df = (shoes_df.groupby(by = ['PRIMARY_STOCKNO','SECONDARY_STOCKNO'], as_index=False)['PRIMARY_STOCKNO'].value_counts()).drop(columns = ['count'])
shoes_nation_level_df

,PRIMARY_STOCKNO,SECONDARY_STOCKNO
0,110036,638113
1,110071,131733
2,110095,1573204
3,110098,638113
4,125014,125033
...,...,...
510957,99869319,99802217
510958,99869319,99802230
510959,99869319,99802240
510960,99869319,99802404


In [40]:
access_df = shoe_acces_sale.reset_index()
access_df = access_df.drop(columns = ['index', 'PRIMARY_UNITS', 'SECONDARY_UNITS',
                              'SECONDARY_ITEM_SIZE', 'SECONDARY_ITEM_SIZE', 'SECONDARY_ITEM_WIDTH',
                              'SECONDARY_SHOE', 'STORE_ZIP',
                              'LATITUDE', 'LONGITUDE'])
access_df.head(10)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,SECONDARY_STOCKNO,PRIMARY_DEPT,SECONDARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA
0,8,12,398562,100,M,19133,3.0,6.0,True,9.0,29.0,MI
1,8,12,532208,110,M,53068,1.0,8.0,True,9.0,29.0,MI
2,8,12,498480,070,M,303,1.0,5.0,True,9.0,29.0,MI
3,8,12,498480,070,M,401,1.0,5.0,True,9.0,29.0,MI
4,8,12,530062,080,M,1725,1.0,15.0,True,9.0,29.0,MI
5,8,12,530062,080,M,53081,1.0,6.0,True,9.0,29.0,MI
6,8,11,498540,040,M,50476,1.0,6.0,True,8.0,15.0,FL
7,8,11,497483,070,M,50476,3.0,6.0,True,8.0,15.0,FL
8,8,11,99398734,050,M,1820,7.0,9.0,True,8.0,15.0,FL
9,8,11,895946,100,M,343,1.0,5.0,True,8.0,15.0,FL


In [41]:
access_nation_level_df = access_df.groupby(by = ['PRIMARY_STOCKNO','SECONDARY_STOCKNO'], as_index=False)['PRIMARY_STOCKNO'].value_counts()
access_nation_level_df

,PRIMARY_STOCKNO,SECONDARY_STOCKNO,count
0,125014,274,2
1,125014,291,5
2,125014,292,2
3,125014,303,1
4,125014,342,1
...,...,...,...
267897,99869319,53051,1
267898,99869319,53083,2
267899,99869319,53109,1
267900,99869319,99460,2


#### Change the store number below. Available store numbers are in the above cell.

In [53]:
shoes_df

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA
128806,8,782,895945,130,M,1.0,True,2.0,9.0,TN
128807,8,782,895946,130,M,1.0,True,2.0,9.0,TN
128808,8,782,497874,070,M,3.0,True,2.0,9.0,TN
128809,8,782,389050,065,M,3.0,True,2.0,9.0,TN
128810,8,782,860431,100,M,3.0,True,2.0,9.0,TN
...,...,...,...,...,...,...,...,...,...,...
1565822,8,782,497832,055,M,1.0,True,2.0,9.0,TN
1565823,8,782,499281,060,M,1.0,True,2.0,9.0,TN
1565824,8,782,499281,060,M,1.0,True,2.0,9.0,TN
1565825,8,782,132026,100,M,3.0,True,2.0,9.0,TN


In [68]:
shoes_nation_level_df = pd.DataFrame(shoes_df['PRIMARY_STOCKNO'].unique())
shoes_nation_level_df.columns = ['PRIMARY_STOCKNO']
shoes_nation_level_df

,PRIMARY_STOCKNO
0,99497860
1,497403
2,1497835
3,398564
4,1399645
...,...
2879,99521194
2880,399921
2881,1480927
2882,1479840


In [74]:
n=1442

#split DataFrame into chunks
list_df = [shoes_nation_level_df[i:i+n] for i in range(0,len(shoes_nation_level_df),n)]

In [75]:
list_df[0]

,PRIMARY_STOCKNO,rec1,rec2,rec3,rec1_count,rec2_count,rec3_count,rec_level
0,99497860,1497860.0,99497858.0,99498066.0,70.0,16.0,14.0,division
1,497403,498066.0,497419.0,398564.0,312.0,92.0,85.0,division
2,1497835,1497800.0,1497878.0,1497721.0,17.0,15.0,13.0,division
3,398564,398913.0,398566.0,1398564.0,3701.0,1237.0,1174.0,division
4,1399645,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
...,...,...,...,...,...,...,...,...
1437,99497873,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438,1362096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,99583115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1440,802409,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
list_df[1]

,PRIMARY_STOCKNO,rec1,rec2,rec3,rec1_count,rec2_count,rec3_count,rec_level
1442,497834,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1443,497778,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444,437184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,497869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,497805,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2879,99521194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2880,399921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2881,1480927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2882,1479840,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
for index, row in list_df[0].iterrows():
    
    if index % 10 == 0:
        print(index)
        
    if len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO)[1][2] > 2:
        shoes_nation_level_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_nation_level_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        shoes_nation_level_df.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][2]
        shoes_nation_level_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_nation_level_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        shoes_nation_level_df.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][2]
        shoes_nation_level_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 2 and shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1] > 2:
        shoes_nation_level_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_nation_level_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        shoes_nation_level_df.loc[index, 'rec3'] = -1
        shoes_nation_level_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_nation_level_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        shoes_nation_level_df.loc[index, 'rec3_count'] = -1
        shoes_nation_level_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 1 and shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0] > 2:
        shoes_nation_level_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_nation_level_df.loc[index, 'rec2'] = -1
        shoes_nation_level_df.loc[index, 'rec3'] = -1
        shoes_nation_level_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_nation_level_df.loc[index, 'rec2_count'] = -1
        shoes_nation_level_df.loc[index, 'rec3_count'] = -1
        shoes_nation_level_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    else:
        shoes_nation_level_df.loc[index, 'rec1'] = -1
        shoes_nation_level_df.loc[index, 'rec2'] = -1
        shoes_nation_level_df.loc[index, 'rec3'] = -1
        shoes_nation_level_df.loc[index, 'rec1_count'] = -1
        shoes_nation_level_df.loc[index, 'rec2_count'] = -1
        shoes_nation_level_df.loc[index, 'rec3_count'] = -1
        shoes_nation_level_df.loc[index, 'rec_level'] = -1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440


In [83]:
shoes_nation_level_df

,PRIMARY_STOCKNO,rec1,rec2,rec3,rec1_count,rec2_count,rec3_count,rec_level
0,99497860,1497860.0,99497858.0,99498066.0,70.0,16.0,14.0,division
1,497403,498066.0,497419.0,398564.0,312.0,92.0,85.0,division
2,1497835,1497800.0,1497878.0,1497721.0,17.0,15.0,13.0,division
3,398564,398913.0,398566.0,1398564.0,3701.0,1237.0,1174.0,division
4,1399645,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
...,...,...,...,...,...,...,...,...
2879,99521194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2880,399921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2881,1480927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2882,1479840,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
shoes_nation_level_df.to_csv('nation_level_shoe_recomendations2.csv', index=False)

In [85]:
shoes_district_level_df = pd.DataFrame(access_df.groupby(by=['PRIMARY_STOCKNO','DISTRICT', 'REGION'], as_index=False).size()).drop(columns=['size'])
shoes_district_level_df

,PRIMARY_STOCKNO,DISTRICT,REGION
0,125014,2.0,1.0
1,125014,3.0,5.0
2,125014,7.0,2.0
3,125014,9.0,2.0
4,125014,11.0,9.0
...,...,...,...
86091,99869319,58.0,8.0
86092,99869319,64.0,3.0
86093,99869319,67.0,8.0
86094,99869319,68.0,1.0


In [ ]:
access_nation_level_df = pd.DataFrame(access_df['PRIMARY_STOCKNO'].unique())
access_nation_level_df.columns = ['PRIMARY_STOCKNO']
access_nation_level_df

In [ ]:
n=1442

#split DataFrame into chunks
access_list_df = [shoes_nation_level_df[i:i+n] for i in range(0,len(shoes_nation_level_df),n)]

In [ ]:
for index, row in access_nation_level_df.iterrows():
    
    if index % 10 == 0:
        print(index)
        
    if len(acces_recomendations2(row.PRIMARY_STOCKNO)[1]) == 3 and acces_recomendations2(row.PRIMARY_STOCKNO)[1][2] > 2:
        access_nation_level_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        access_nation_level_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        access_nation_level_df.loc[index, 'rec3'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][2]
        access_nation_level_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        access_nation_level_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        access_nation_level_df.loc[index, 'rec3_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][2]
        shoes_nation_level_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(acces_recomendations2(row.PRIMARY_STOCKNO)[1]) == 2 and acces_recomendations2(row.PRIMARY_STOCKNO)[1][1] > 2:
        shoes_nation_level_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_nation_level_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        shoes_nation_level_df.loc[index, 'rec3'] = -1
        shoes_nation_level_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_nation_level_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        shoes_nation_level_df.loc[index, 'rec3_count'] = -1
        shoes_nation_level_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(acces_recomendations2(row.PRIMARY_STOCKNO)[1]) == 1 and acces_recomendations2(row.PRIMARY_STOCKNO)[1][0] > 2:
        shoes_nation_level_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_nation_level_df.loc[index, 'rec2'] = -1
        shoes_nation_level_df.loc[index, 'rec3'] = -1
        shoes_nation_level_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_nation_level_df.loc[index, 'rec2_count'] = -1
        shoes_nation_level_df.loc[index, 'rec3_count'] = -1
        shoes_nation_level_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    else:
        shoes_nation_level_df.loc[index, 'rec1'] = -1
        shoes_nation_level_df.loc[index, 'rec2'] = -1
        shoes_nation_level_df.loc[index, 'rec3'] = -1
        shoes_nation_level_df.loc[index, 'rec1_count'] = -1
        shoes_nation_level_df.loc[index, 'rec2_count'] = -1
        shoes_nation_level_df.loc[index, 'rec3_count'] = -1
        shoes_nation_level_df.loc[index, 'rec_level'] = -1

In [ ]:
shoes_nation_level_df.to_csv('nation_level_access_recomendations.csv', index=False)

### The below for loop uses the shoe_recomendations2 function on each primary_stockno. It starts by looking at the store level. If it does not find 3 recomendations (with more than 2 of that combo purchased) it will move to the highest level. If it can't find a recomendation for a shoe it will return -1.<br><br>
#### This loop is pretty inefficient and will take an extremely long time to complete the entire sales table (1.76 million rows). The current version is running on 11 rows and takes approximately 40 seconds to complete. This would equate to 81 days for it to run through all 1.76 million rows.

In [75]:
for index, row in shoes_nation_level_df.iterrows():
    
    if index % 10 == 0:
        print(index)
        
    if len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                            [row.STORE_STATE_ALPHA], [row.STORE])[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION],
                                                                                                    [row.DISTRICT],[row.STORE_STATE_ALPHA],
                                                                                                    [row.STORE])[1][2] > 2:
                            
        shoes_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][0]
        shoes_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][1]
        shoes_df.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][2]
        shoes_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][0]
        shoes_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][1]
        shoes_df.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][2]
        shoes_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                            [row.STORE_STATE_ALPHA])[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                                                                                       [row.STORE_STATE_ALPHA])[1][2] > 2:
        shoes_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][0]
        shoes_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][1]
        shoes_df.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][2]
        shoes_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][0]
        shoes_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][1]
        shoes_df.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][2]
        shoes_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT])[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                                       [row.REGION],
                                                                                                                       [row.DISTRICT])[1][2] > 2:
        shoes_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][0]
        shoes_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][1]
        shoes_df.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][2]
        shoes_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][0]
        shoes_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][1]
        shoes_df.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][2]
        shoes_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                       [row.REGION])[1][2] > 2:
        shoes_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][0]
        shoes_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][1]
        shoes_df.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][2]
        shoes_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][0]
        shoes_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][1]
        shoes_df.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][2]
        shoes_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 3:
        shoes_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        shoes_df.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][2]
        shoes_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        shoes_df.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][2]
        shoes_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 2:
        shoes_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_df.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        shoes_df.loc[index, 'rec3'] = -1
        shoes_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_df.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        shoes_df.loc[index, 'rec3_count'] = -1
        shoes_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 1:
        shoes_df.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        shoes_df.loc[index, 'rec2'] = -1
        shoes_df.loc[index, 'rec3'] = -1
        shoes_df.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        shoes_df.loc[index, 'rec2_count'] = -1
        shoes_df.loc[index, 'rec3_count'] = -1
        shoes_df.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    else:
        shoes_df.loc[index, 'rec1'] = -1
        shoes_df.loc[index, 'rec2'] = -1
        shoes_df.loc[index, 'rec3'] = -1
        shoes_df.loc[index, 'rec1_count'] = -1
        shoes_df.loc[index, 'rec2_count'] = -1
        shoes_df.loc[index, 'rec3_count'] = -1
        shoes_df.loc[index, 'rec_level'] = -1

209750
209760
209770
209780
209790
209800
209810
210100
210110
210120
210130
210140
210150
210160
210170
210180
210190
210200
210210
210220
210230
223510
223520
223530
223540
223550
223560
223730
223740
223750
223760
223770
223780
223790
223800
223810
223820
223830
223840
223850
223860
223870
223880
223890
223900
223910
223920
223930
223940
223950
223960
223970
223980
223990
224000
224010
224020
224030
224040
224050
224060
224070
224080
224090
224100
224110
224120
224130
224140
224150
224160
224170
224180
224190
224200
224210
224220
224230
224240
224250
224260
224270
224280
224290
224300
224390
224400
224410
224420
224430
224440
224450
224460
269120
269130
269140
269150
269160
269170
269330
269340
269350
269360
269370
269380
269390
269400
269410
269430
269440
269450
269460
269470
269480
269490
269500
269510
269520
269530
269540
269550
269560
918510
918520
918530
918540
918720
918730
918740
918750
918760
918770
918780
918790
918800
918810
918820
918830
918840
918850
918860
918870
918880

In [32]:
shoes_df

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA,rec1,rec2,rec3,rec1_count,rec2_count,rec3_count,rec_level
73000,8,492,860428,080,M,3.0,True,2.0,19.0,TN,860427.0,860461.0,802403.0,53.0,39.0,36.0,division
73001,8,492,531963,080,M,1.0,True,2.0,19.0,TN,532106.0,532211.0,532107.0,104.0,70.0,63.0,division
73002,8,492,582113,080,M,3.0,True,2.0,19.0,TN,581628.0,581627.0,581905.0,108.0,66.0,46.0,division
73003,8,492,581724,060,M,3.0,True,2.0,19.0,TN,896070.0,581797.0,99581703.0,12.0,7.0,5.0,district
73004,8,492,99399904,100,M,7.0,True,2.0,19.0,TN,99398561.0,1399904.0,1398571.0,6.0,4.0,3.0,district
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536615,8,492,399873,060,M,3.0,True,2.0,19.0,TN,399698.0,398564.0,389030.0,90.0,70.0,56.0,division
1536616,8,492,382088,120,M,1.0,True,2.0,19.0,TN,382042.0,382087.0,382089.0,86.0,74.0,72.0,division
1536617,8,492,399645,050,M,3.0,True,2.0,19.0,TN,398230.0,398564.0,398274.0,4.0,3.0,3.0,district
1536618,8,492,399646,050,M,3.0,True,2.0,19.0,TN,398564.0,398329.0,398913.0,5.0,3.0,3.0,district


In [76]:
shoes_df.to_csv('store1055_1111_1245_shoe_recomendations.csv', index=False)

In [ ]:
for index, row in access_df.iterrows():
    
    if index % 10 == 0:
        print(index)
        
    if len(acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                            [row.STORE_STATE_ALPHA], [row.STORE])[1]) == 3 and acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION],
                                                                                                    [row.DISTRICT],[row.STORE_STATE_ALPHA],
                                                                                                    [row.STORE])[1][2] > 1:
                            
        access_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][0]
        access_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][1]
        access_df.loc[index, 'rec3'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][2]
        access_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][0]
        access_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][1]
        access_df.loc[index, 'rec3_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][2]
        access_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[2]
        
    elif len(acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                            [row.STORE_STATE_ALPHA])[1]) == 3 and acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                                                                                       [row.STORE_STATE_ALPHA])[1][2] > 1:
        access_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][0]
        access_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][1]
        access_df.loc[index, 'rec3'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][2]
        access_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][0]
        access_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][1]
        access_df.loc[index, 'rec3_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][2]
        access_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[2]
        
    elif len(acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT])[1]) == 3 and acces_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                                       [row.REGION],
                                                                                                                       [row.DISTRICT])[1][2] > 1:
        access_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][0]
        access_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][1]
        access_df.loc[index, 'rec3'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][2]
        access_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][0]
        access_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][1]
        access_df.loc[index, 'rec3_count'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][2]
        access_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[2]
        
    elif len(acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]) == 3 and acces_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                       [row.REGION])[1][2] > 1:
        access_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][0]
        access_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][1]
        access_df.loc[index, 'rec3'] = acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][2]
        access_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][0]
        access_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][1]
        access_df.loc[index, 'rec3_count'] = acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][2]
        access_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 3:
        access_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        access_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        access_df.loc[index, 'rec3'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][2]
        access_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        access_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        access_df.loc[index, 'rec3_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][2]
        access_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(acces_recomendations2(row.PRIMARY_STOCKNO)[1]) == 2:
        access_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        access_df.loc[index, 'rec2'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        access_df.loc[index, 'rec3'] = -1
        access_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        access_df.loc[index, 'rec2_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        access_df.loc[index, 'rec3_count'] = -1
        access_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(acces_recomendations2(row.PRIMARY_STOCKNO)[1]) == 1:
        access_df.loc[index, 'rec1'] = acces_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        access_df.loc[index, 'rec2'] = -1
        access_df.loc[index, 'rec3'] = -1
        access_df.loc[index, 'rec1_count'] = acces_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        access_df.loc[index, 'rec2_count'] = -1
        access_df.loc[index, 'rec3_count'] = -1
        access_df.loc[index, 'rec_level'] = acces_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    else:
        access_df.loc[index, 'rec1'] = -1
        access_df.loc[index, 'rec2'] = -1
        access_df.loc[index, 'rec3'] = -1
        access_df.loc[index, 'rec1_count'] = -1
        access_df.loc[index, 'rec2_count'] = -1
        access_df.loc[index, 'rec3_count'] = -1
        access_df.loc[index, 'rec_level'] = -1

177610
177620
177630
177850
177860
177870
177880
177890
177900
190060
190070
190080
190090
190100
190110
190120
190130
190260
190270
190280
190290
190300
190310
190320
190330
190340
190350
190360
190370
190380
190390
190400
190410
190420
190430
190440
190450
190460
190470
190480
190490
190500
190510
190520
190530
190540
190550
190560
190570
190580
190590
190600
190610
190620
190630
190640
190650
190660
190670
190680
190690
190700
190790
190800
190810
190820
228940
228950
229120
229130
229140
229150
229160
229170
229180
229190
229200
229210
229220
229230
229240
229250
229260
229270
229280
229290
229300
229310
229320
717890
717900
718060
718070
718080
718090
718100
718120
718130
718140
718150
718160
718170
718180
718190
718200
718210
718220
718230
739390
739400
739410
739420
739430
739440
739450
739460
739580
739590
739600
739610
739620
739630
739640
739650
739660
739670
739680
739690
739700
739710
739720
739730


In [48]:
access_df

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA,rec1,rec2,rec3,rec1_count,rec2_count,rec3_count,rec_level
83006,8,583,521219,070,M,3.0,True,2.0,9.0,TN,99512.0,99499.0,99468.0,8.0,7.0,6.0,division
83007,8,583,521219,070,M,3.0,True,2.0,9.0,TN,99512.0,99499.0,99468.0,8.0,7.0,6.0,division
83008,8,583,521219,080,M,3.0,True,2.0,9.0,TN,99512.0,99499.0,99468.0,8.0,7.0,6.0,division
83009,8,583,521219,080,M,3.0,True,2.0,9.0,TN,99512.0,99499.0,99468.0,8.0,7.0,6.0,division
83010,8,583,437129,090,M,3.0,True,2.0,9.0,TN,1639.0,401.0,292.0,8.0,6.0,3.0,division
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161413,8,583,569905,080,M,3.0,True,2.0,9.0,TN,5754.0,291.0,4127.0,12.0,11.0,4.0,state
1161414,8,583,569905,080,M,3.0,True,2.0,9.0,TN,5754.0,291.0,4127.0,12.0,11.0,4.0,state
1161415,8,583,569134,080,M,3.0,True,2.0,9.0,TN,5754.0,5753.0,434.0,144.0,130.0,77.0,division
1161416,8,583,362035,080,M,3.0,True,2.0,9.0,TN,1639.0,403.0,402.0,37.0,24.0,15.0,division


In [ ]:
access_df.to_csv('store1055_1111_1245_access_recomendations.csv', index=False)